#### Backbone(BERT)은 freeze, classifier + LoRA 모듈만 학습

✅ 라이브러리 & 환경 설정

In [1]:
import torch, random
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader
from peft import LoraConfig, get_peft_model, TaskType

import matplotlib.pyplot as plt
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    get_scheduler,
)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("✅ Device:", device)

✅ Device: mps


✅ 데이터셋 로드 & 8:1:1 분할

In [2]:
# SST-2 원본 로드
dataset = load_dataset("glue", "sst2")
train_data = dataset["train"]

# 데이터 변환
train_data = train_data.to_pandas()

# 8:1:1 분할
train_split, temp_split = train_test_split(train_data, test_size=0.2, random_state=42)
val_split, test_split = train_test_split(temp_split, test_size=0.5, random_state=42)

# Hugging Face Dataset 형태로 변환
train_dataset = Dataset.from_dict(train_split)
val_dataset   = Dataset.from_dict(val_split)
test_dataset  = Dataset.from_dict(test_split)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

Train: 53879, Val: 6735, Test: 6735


✅ 토크나이저 및 전처리

In [3]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 전처리 함수
def preprocess(batch):
    return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=128)

# 모든 split에 적용
datasets_encoded = {
    "train": train_dataset.map(preprocess, batched=True),
    "validation": val_dataset.map(preprocess, batched=True),
    "test": test_dataset.map(preprocess, batched=True), 
}

# 데이터셋 텐서 형식 설정
for name in datasets_encoded:
    datasets_encoded[name] = datasets_encoded[name].rename_column("label", "labels")
    datasets_encoded[name].set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"]
    )

/opt/anaconda3/envs/bert-project/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/53879 [00:00<?, ? examples/s]

Map:   0%|          | 0/6735 [00:00<?, ? examples/s]

Map:   0%|          | 0/6735 [00:00<?, ? examples/s]

✅ 모델 구성 (Encoder Freeze)

Query, Value에만 Lora적용

In [4]:
def init_model(dropout, r=8, alpha=16, target_modules=["query", "value"]):
    # 1) 베이스 모델 로드
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2
    )

    # 2) dropout 반영
    if hasattr(model, "dropout"):
        model.dropout.p = dropout

    # 3) 전체 freeze (BERT backbone)
    for p in model.parameters():
        p.requires_grad = False

    # 4) classifier는 항상 학습
    for name, p in model.named_parameters():
        if name.startswith("classifier."):
            p.requires_grad = True

    # 5) LoRA 설정
    lora_cfg = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=target_modules,   # attention의 Q,V
        lora_dropout=0.05,
        task_type=TaskType.SEQ_CLS,
        bias="none"
    )

    # 6) LoRA 적용
    model = get_peft_model(model, lora_cfg)


    model.to(device)
    return model

✅ Optimizer & Loss 구성

In [5]:
def configure_optimizer(model, lr, num_epochs, train_loader):

    # classifier.parameters만 수정
    # 🔹 requires_grad=True 인 파라미터만 선택
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    
    optimizer = torch.optim.AdamW(trainable_params, lr=lr)
    
    # 학습률 스케줄러 설정 
    num_training_steps = num_epochs * len(train_loader)
    scheduler = get_scheduler("linear", optimizer=optimizer,
                              num_warmup_steps=0, num_training_steps=num_training_steps)
    # 손실 함수 설정
    criterion = torch.nn.CrossEntropyLoss()
    
    return optimizer, scheduler, criterion

✅ 학습 루프

In [6]:
def train_model(model, optimizer, scheduler, train_loader, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # forward
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Avg Loss: {total_loss/len(train_loader):.4f}")

✅ 평가 함수

In [7]:
def evaluate(model, data_loader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # 🔹 예측
            logits = model(**batch).logits
            pred = logits.argmax(-1)

            preds.extend(pred.cpu().numpy())
            labels.extend(batch["labels"].cpu().numpy())

    # 🔹 주요 지표 계산
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    rec = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    print(f"📊 Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")
    return {
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

✅ Random Search 설정

In [8]:
# 실험할 LoRA rank 값들
r_list = [4, 8, 16]

# 랜덤 서치용 하이퍼파라미터 공간
search_space = {
    "lr":        [2e-5, 3e-5, 5e-5],
    "batch_size":[16, 32],
    "dropout":   [0.1, 0.2],
    "epochs":    [2, 3, 4],
}

n_trials_per_r = 3   # r마다 몇 번씩 돌려볼지

✅ Random Search Training

In [9]:
results = []
best_val_acc = 0.0
best_model = None
best_params = None

for r in r_list:
    print("\n" + "="*50)
    print(f"🔷 Running experiments for LoRA rank r = {r}")
    print("="*50)

    for trial in range(n_trials_per_r):
        # 🔹 랜덤 하이퍼파라미터 선택
        params = {k: random.choice(v) for k, v in search_space.items()}
        print(f"\n🎯 [r={r}] Trial {trial+1}/{n_trials_per_r} | Params = {params}")

        # 🔹 모델 초기화 (LoRA 적용)
        model = init_model(params["dropout"], r=r)

        train_loader = DataLoader(
            datasets_encoded["train"], 
            batch_size=params["batch_size"], 
            shuffle=True
        )
        val_loader  = DataLoader(datasets_encoded["validation"], batch_size=64)
        test_loader = DataLoader(datasets_encoded["test"], batch_size=64)

        # 🔹 Optimizer & Scheduler 구성
        optimizer, scheduler, criterion = configure_optimizer(
            model, params["lr"], params["epochs"], train_loader
        )

        # 🔹 학습
        train_model(model, optimizer, scheduler, train_loader, num_epochs=params["epochs"])

        # 🔹 성능 평가
        val_metrics  = evaluate(model, val_loader)
        test_metrics = evaluate(model, test_loader)

        print(f"📌 [r={r}] Val Acc: {val_metrics['acc']*100:.2f}% | Test Acc: {test_metrics['acc']*100:.2f}%")

        # 🔹 결과 저장
        results.append({
            "r": r,
            "trial": trial + 1,
            "lr": params["lr"],
            "batch_size": params["batch_size"],
            "dropout": params["dropout"],
            "epochs": params["epochs"],
            "val_acc": val_metrics["acc"],
            "val_prec": val_metrics["prec"],
            "val_rec": val_metrics["rec"],
            "val_f1": val_metrics["f1"],
            "test_acc": test_metrics["acc"],
            "test_prec": test_metrics["prec"],
            "test_rec": test_metrics["rec"],
            "test_f1": test_metrics["f1"],
        })

        # 🔹 베스트 모델 저장
        if val_metrics["acc"] > best_val_acc:
            best_val_acc = val_metrics["acc"]
            best_model = model
            best_params = {**params, "r": r}

            save_dir = f"./lora_best_r{r}"
            model.save_pretrained(save_dir)
            tokenizer.save_pretrained(save_dir)

            print(f"💾 Best model updated! r={r}, Val={best_val_acc*100:.2f}% (Saved → {save_dir})")


🔷 Running experiments for LoRA rank r = 4

🎯 [r=4] Trial 1/3 | Params = {'lr': 3e-05, 'batch_size': 32, 'dropout': 0.2, 'epochs': 2}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/2: 100%|██████████| 1684/1684 [18:28<00:00,  1.52it/s]


Epoch 1 | Avg Loss: 0.3965


Epoch 2/2: 100%|██████████| 1684/1684 [18:32<00:00,  1.51it/s]


Epoch 2 | Avg Loss: 0.2898
📊 Acc: 0.8851 | Prec: 0.9055 | Rec: 0.8878 | F1: 0.8966
📊 Acc: 0.8944 | Prec: 0.9118 | Rec: 0.8990 | F1: 0.9054
📌 [r=4] Val Acc: 88.51% | Test Acc: 89.44%
💾 Best model updated! r=4, Val=88.51% (Saved → ./lora_best_r4)

🎯 [r=4] Trial 2/3 | Params = {'lr': 3e-05, 'batch_size': 16, 'dropout': 0.2, 'epochs': 2}


...

Epoch 1 | Avg Loss: 0.4305


Epoch 2/3: 100%|██████████| 1684/1684 [18:21<00:00,  1.53it/s]


Epoch 2 | Avg Loss: 0.2978


Epoch 3/3: 100%|██████████| 1684/1684 [18:24<00:00,  1.52it/s]


Epoch 3 | Avg Loss: 0.2827
📊 Acc: 0.8857 | Prec: 0.9021 | Rec: 0.8931 | F1: 0.8976
📊 Acc: 0.8941 | Prec: 0.9029 | Rec: 0.9093 | F1: 0.9061
📌 [r=16] Val Acc: 88.57% | Test Acc: 89.41%


📊 결과 정리 및 저장

🔹 최종 결과 요약


In [12]:
print("\n📊 Random Search Summary (Sorted by Validation Accuracy):")
print(df.sort_values("val_acc", ascending=False))
print("\n🏆 Best Trial Parameters:")
print(best_params, f"Val Acc: {best_val_acc*100:.2f}%")


📊 Random Search Summary (Sorted by Validation Accuracy):
    r  trial       lr  batch_size  dropout  epochs   val_acc  val_prec  \
4   8      2  0.00005          32      0.1       4  0.903192  0.921294   
2   4      3  0.00005          32      0.1       3  0.898738  0.918152   
7  16      2  0.00005          16      0.2       2  0.895917  0.910157   
5   8      3  0.00003          32      0.1       3  0.892502  0.908507   
3   8      1  0.00003          32      0.2       4  0.892353  0.907610   
6  16      1  0.00003          32      0.2       3  0.889681  0.906074   
1   4      2  0.00003          16      0.2       2  0.887751  0.907058   
8  16      3  0.00002          32      0.2       3  0.885672  0.902139   
0   4      1  0.00003          32      0.2       2  0.885078  0.905508   

    val_rec    val_f1  test_acc  test_prec  test_rec   test_f1  
4  0.904711  0.912927  0.913140   0.927083  0.917526  0.922280  
2  0.899682  0.908824  0.907350   0.922439  0.911710  0.917043  
7  0.9

In [16]:
df = pd.DataFrame(results)
df.to_csv("lora_finetune_random_search_tvt.csv", index=False)

In [17]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,}/{total:,} ({100*trainable/total:.6f}%)")

Trainable: 591,362/110,075,140 (0.537235%)


In [18]:
trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Optimizer params: {sum(p.numel() for p in trainable_params):,}")

Optimizer params: 591,362


In [19]:
for name, param in model.named_parameters():
    if param.grad is not None:
        print(name, param.grad.abs().mean().item())

In [20]:
print(best_model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defaul

best modle torch로 저장

In [21]:
torch.save(best_model.state_dict(), "lora_model.pt")
print("💾 Saved best model weights as lora_model.pt")

💾 Saved best model weights as lora_model.pt
